In [11]:
import pandas as pd 
import string
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer

In [12]:
df = pd.read_csv('Data/manually_cleaned/train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to shelter in place are be...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
df.drop(['location', 'id'], axis=1, inplace=True)
df.fillna('none', inplace=True)
df.head()

,keyword,text,target
0,none,Our Deeds are the Reason of this #earthquake M...,1
1,none,Forest fire near La Ronge Sask. Canada,1
2,none,All residents asked to shelter in place are be...,1
3,none,"13,000 people receive #wildfires evacuation or...",1
4,none,Just got sent this photo from Ruby #Alaska as ...,1


In [14]:
#Add back spaces in keywords
keywords = df['keyword'].to_list()
clean_keywords = [word.replace('%20', ' ') for word in keywords]
df['keyword'] = clean_keywords
df.head()

,keyword,text,target
0,none,Our Deeds are the Reason of this #earthquake M...,1
1,none,Forest fire near La Ronge Sask. Canada,1
2,none,All residents asked to shelter in place are be...,1
3,none,"13,000 people receive #wildfires evacuation or...",1
4,none,Just got sent this photo from Ruby #Alaska as ...,1


In [15]:
data = df
data.head()

,keyword,text,target
0,none,Our Deeds are the Reason of this #earthquake M...,1
1,none,Forest fire near La Ronge Sask. Canada,1
2,none,All residents asked to shelter in place are be...,1
3,none,"13,000 people receive #wildfires evacuation or...",1
4,none,Just got sent this photo from Ruby #Alaska as ...,1


In [16]:
#Use Pandas and RegEx to copy hashtags, mentions, and urls and store them each in a list in each their separate columns
data["hashtags"] = data['text'].str.findall(r'([#]\S+)')
data["mentions"] = data['text'].str.findall(r'([@]\S+)')
data["urls"] = data['text'].str.findall(r'(http:\/\/\S+)')
#Effectively deletes mentions and urls with Pandas and Regex by replacing them with an empty string as they are probably not particularly useful in NLP for Tweets
data["text"] = data["text"].str.replace('([@]\S+)', '', regex=True) #deletes mentions
data["text"] = data["text"].str.replace('((https?:\/\/)\S+)', '', regex=True) #deletes URLs
#Replaces the # symbol with an empty space to turn hashtags into regular words
data["text"] = data["text"].str.replace('[#:\n]', '', regex=True) #deletes the hashtage symbol only
#Replaces non ASCII words and symbols like: Â‰Ã›Ã with blank spaces
data["text"] = data["text"].str.replace('[^\x00-\x7F]+', '', regex=True) #deletes non English alphabet characters#Use Pandas and RegEx to copy hashtags, mentions, and urls and store them each in a list in each their separate columns
data.head()

,keyword,text,target,hashtags,mentions,urls
0,none,Our Deeds are the Reason of this earthquake Ma...,1,[#earthquake],[],[]
1,none,Forest fire near La Ronge Sask. Canada,1,[],[],[]
2,none,All residents asked to shelter in place are be...,1,[],[],[]
3,none,"13,000 people receive wildfires evacuation ord...",1,[#wildfires],[],[]
4,none,Just got sent this photo from Ruby Alaska as s...,1,"[#Alaska, #wildfires]",[],[]


In [17]:
#Further text cleaning
data['text'] = data['text'].str.lower()
data['text'] = data['text'].str.translate(str.maketrans('', '', string.punctuation))
data['text'] = data['text'].str.rstrip(" ")
data['text'] = data['text'].str.lstrip(" ")
#Lower Casing hashtags
hashtags = data['hashtags'].tolist()
lower_tags = []
for list in hashtags:
    new_list = []
    if len(list) > 0:
        for tag in list:
            word = tag[1::]
            lower_tag = word.lower()
            new_list.append(lower_tag)
    else:
        new_list.append('none')
    lower_tags.append(new_list)
#Replacing null mentions and urls
mentions = data['mentions'].tolist()
clean_mentions = []
for list in mentions:
    if len(list) == 0:
        clean_mentions.append(['none'])
    else:
        clean_mentions.append(list)
urls = data['urls'].tolist()
clean_urls = []
for list in urls:
    if len(list) == 0:
        clean_urls.append(['none'])
    else:
        clean_urls.append(list)
data['mentions'] = clean_mentions
data['urls'] = clean_urls 
data['hashtags'] = lower_tags
data.head()

,keyword,text,target,hashtags,mentions,urls
0,none,our deeds are the reason of this earthquake ma...,1,[earthquake],[none],[none]
1,none,forest fire near la ronge sask canada,1,[none],[none],[none]
2,none,all residents asked to shelter in place are be...,1,[none],[none],[none]
3,none,13000 people receive wildfires evacuation orde...,1,[wildfires],[none],[none]
4,none,just got sent this photo from ruby alaska as s...,1,"[alaska, wildfires]",[none],[none]


In [18]:
contractions = {
"<3": "love", 
"aint": "am not",
"arent": "are not",
"cant": "cannot",
"cantve": "cannot have",
"cause": "because",
"cuz": "because",
"couldve": "could have",
"couldnt": "could not",
"couldntve": "could not have",
"didnt": "did not",
"doesnt": "does not",
"dont": "do not",
"hadnt": "had not",
"hadntve": "had not have",
"hasnt": "has not",
"havent": "have not",
"hed": "he would",
"hedve": "he would have",
"hell": "he will",
"hes": "he is",
"howd": "how did",
"howll": "how will",
"hows": "how is",
"id": "i would",
"ill": "i will",
"im": "i am",
"ive": "i have",
"isnt": "is not",
"itd": "it would",
"itll": "it will",
"its": "it is",
"lets": "let us",
"luv": "love",
"maam": "madam",
"maynt": "may not",
"mightve": "might have",
"mightnt": "might not",
"mustve": "must have",
"mustnt": "must not",
"neednt": "need not",
"oughtnt": "ought not",
"shant": "shall not",
"shant": "shall not",
"shed": "she would",
"shell": "she will",
"shes": "she is",
"shouldve": "should have",
"shouldnt": "should not",
"thatd": "that would",
"thats": "that is",
"thered": "there had",
"theres": "there is",
"theyd": "they would",
"theyll": "they will",
"theyre": "they are",
"theyve": "they have",
"w/": "with",
"w/o": "without",
"wasnt": "was not",
"wat": "what",
"wed": "we would",
"well": "we will",
"were": "we are",
"weve": "we have",
"werent": "were not",
"whatll": "what will",
"whatre": "what are",
"whats": "what is",
"whatve": "what have",
"whered": "where did",
"wheres": "where is",
"wholl": "who will",
"whos": "who is",
"wont": "will not",
"wouldnt": "would not",
"youd": "you would",
"youll": "you will",
"youre": "you are",
"thx"   : "thanks"
}

In [20]:
#Clean tweet text
#Set a variable to call the Porter Stemmer algorithm
porter = PorterStemmer()
tweets = data['text'].tolist()
no_contractions = []
for tweet in tweets:
    #Set tweet to list of words for individualized cleaning
    words = tweet.split(" ")
    clean_words = []
    for word in words:
        if word in contractions.keys():
            clean_words.append(contractions[word])
        else:
            clean_words.append(word)
    stemmed_words = [porter.stem(word) for word in clean_words]
    no_contractions.append(" ".join(stemmed_words))
data['text'] = no_contractions
data.head() 

,keyword,text,target,hashtags,mentions,urls
0,none,our deed are the reason of thi earthquak may a...,1,[earthquake],[none],[none]
1,none,forest fire near la rong sask canada,1,[none],[none],[none]
2,none,all resid ask to shelter in place are be notif...,1,[none],[none],[none]
3,none,13000 peopl receiv wildfir evacu order in cali...,1,[wildfires],[none],[none]
4,none,just got sent thi photo from rubi alaska as sm...,1,"[alaska, wildfires]",[none],[none]


In [21]:
data.to_csv('Data/manually_cleaned/train_regex_3.0.csv')